## Custom environment using private Python packages from GitHub

In this tutorial, we will walk through the stages of creating a custom environment that uses a private Python package housed in a private GitHub repository.

Pre-requisites:
- A read-only PAT token for the private repository of choice,
- A key vault where the PAT token will be housed,
- A managed identity with the RBAC permission to read the secret
- The managed identity will then need to be assigned to the compute instance

References:
- https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-private-python-packages?view=azureml-api-1
- https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.ipynb

**Learning Objectives** - By the end of this tutorial, you should be able to use Azure Machine Learning (Azure ML) to create a custom environment that includes dependencies held in private repositories on GitHub.

## Step 1: Getting the workspace handle and installing the required packages

In [1]:
!pip install azure-identity 
!pip install azure-keyvault-secrets #Key Vault Python SDK

In [2]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#Enter details of your Azure Machine Learning workspace
subscription_id = '876b91eb-54d6-4433-af3b-5c9914d5ccea'
resource_group = 'ej_vision_playground'
workspace = 'ej-workshop-workspace'

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

## Step 2: Reading the PAT token from the Key Vault

In [4]:
from azure.identity import ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient

credential = ManagedIdentityCredential(client_id="4170009e-5270-484e-b7da-d788b648b7f3")
KEY_VAULT_NAME = "ej-kv-test1"
SECRET_NAME = "ej-pandas-gh"

secret_client = SecretClient(vault_url=f"https://{KEY_VAULT_NAME}.vault.azure.net", credential=credential)
retrieved_secret = secret_client.get_secret(SECRET_NAME)
secret_value = retrieved_secret.value

## Step 3: Creating the workspace connection to the GitHub respository

In [ ]:
from azure.ai.ml.entities import WorkspaceConnection
from azure.ai.ml.entities import PatTokenConfiguration

credentials = PatTokenConfiguration(pat=secret_value)

ws_connection = WorkspaceConnection(
    name="ej-pandas-conn",
    target="https://github.com/ejones18/ej-pandas",
    type="PythonFeed",
    credentials=credentials,
)

ml_client.connections.create_or_update(ws_connection)

## Step 4: Create the custom environment

In [ ]:
from azure.ai.ml.entities import Environment, BuildContext

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="./requirements.yml",
    name="ej-test",
    version="2"
)
ml_client.environments.create_or_update(env_docker_conda)

## Step 5: Submit a job to test the new custom environment

In [ ]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "jonesethan3"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

In [ ]:
from azure.ai.ml import command

# define the command
command_job = command(
    code="./src",
    command="python main.py",
    environment="ej-test:2",
    compute=cpu_compute_target,
)

In [ ]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url